In [5]:
import pandas as pd
import seaborn as sns
import numpy as np
import urllib.request
import tarfile


from tqdm import tqdm
import requests

def track_dl(mytar,url):
    response = requests.get(url, stream=True)
    with open(mytar, "wb") as handle:
        for data in tqdm(response.iter_content()):
            handle.write(data)
        


def get_data2(path,url,file):
    try:
        df = pd.read_csv(path)
    except:
#        urllib.request.urlretrieve(url, "my.tar.gz")
        track_dl(url, "my.tar.gz")
        tf = tarfile.open("my.tar.gz")
        tf.extract(file)
        df = pd.read_table(file, sep="\t")
        df.to_csv(path)
    return df

get_data("luad.tsv", 'http://download.cbioportal.org/luad_tcga_pan_can_atlas_2018.tar.gz',"data_RNA_Seq_v2_expression_median.txt")
get_data("lusc.tsv", 'http://download.cbioportal.org/lusc_tcga_pan_can_atlas_2018.tar.gz',"data_RNA_Seq_v2_expression_median.txt")

,Hugo_Symbol,Entrez_Gene_Id,TCGA-18-3406-01,TCGA-18-3407-01,TCGA-18-3408-01,TCGA-18-3410-01,TCGA-18-3411-01,TCGA-18-3412-01,TCGA-18-3414-01,TCGA-18-3415-01,...,TCGA-NK-A5CT-01,TCGA-NK-A5CX-01,TCGA-NK-A7XE-01,TCGA-O2-A52N-01,TCGA-O2-A52Q-01,TCGA-O2-A52S-01,TCGA-O2-A52V-01,TCGA-O2-A52W-01,TCGA-O2-A5IB-01,TCGA-XC-AA0X-01
0,NaN,100130426,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,NaN,100133144,75.3622,14.4148,8.4888,16.6653,12.9418,47.5782,12.3168,55.1387,...,11.4308,40.3766,20.8723,30.2240,4.2895,19.9503,30.0872,53.6593,72.5666,44.7501
2,NaN,100134869,25.4252,6.9933,3.3754,19.5722,14.4142,20.8881,25.7116,39.0809,...,2.2967,17.3322,15.7901,4.2022,3.6632,47.1026,15.2957,33.3907,42.1832,23.6304
3,NaN,10357,279.9760,198.2540,254.5930,200.8030,277.6670,394.3620,205.8700,162.0410,...,97.8986,142.3690,133.6910,131.5630,153.5980,176.7180,188.7210,260.3330,160.1620,76.6488
4,NaN,10431,1510.2400,968.6010,1474.1200,1156.0600,1119.5800,861.5020,1030.4600,1931.2100,...,1196.9400,935.5670,2257.2700,607.6500,1103.4500,1188.3300,1248.0300,789.3610,460.8630,805.9510
5,NaN,136542,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
6,NaN,155060,23.6220,301.6180,49.2362,120.9230,33.3903,60.6416,204.1720,35.8382,...,275.0790,226.9230,502.5280,789.6170,162.0380,226.8210,147.4940,854.3790,569.2230,311.6270
7,NaN,26823,0.0000,0.0000,1.7796,0.0000,0.4023,0.0000,0.3692,0.0000,...,0.0000,0.0000,0.6321,2.7322,0.4970,1.6556,0.4202,0.0000,1.3312,0.5220
8,NaN,280660,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,NaN,317712,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
